<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [8]:
%%capture --no-stderr
# %pip install pyautogen~=0.2.0b4

In [9]:
%pip install nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("vader_lexicon")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/vscode/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [10]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
         "model": ["gpt-3.5-turbo-16k-0613"],
        # "model": ["gpt-3.5-turbo"],
    },
)

# Import Settings

In [11]:
import json

gpt_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,  # Controls randomness of responses
    "config_list": config_list_gpt4,  # Presumably a list of additional GPT-3.5 configurations
    "timeout": 120,  # Maximum time for the agent to respond
}

# Load agent configurations from the JSON file
with open('agents_config.json', 'r') as file:
    agents_config = json.load(file)


## Construct Agents

In [12]:
# Initialize agents using the configurations from the JSON file
Agents_1 = []
Agents_2 = []
Agents_3 = []

for key, config in agents_config.items():
    # Extracting relevant details from the new JSON structure
    role = config['Role']
    demographic = config['Demographic']
    opinions = config['Opinions']
    task = config['Task']
    format_requirement = config['Format']

    # Constructing a system message based on the new JSON structure
    system_message_1 = f"[Role] {role}\n[Task] {task}\n[Format] {format_requirement}"
    system_message_2 = f"[Role] {role}\n[Opinions] {opinions}\n[Task] {task}\n[Format] {format_requirement}"
    system_message_3 = f"[Role] {role}\n[Demographic] {demographic}\n[Opinions] {opinions}\n[Task] {task}\n[Format] {format_requirement}"

    # Creating the agent with the new system message
    agent_1 = autogen.AssistantAgent(
        name=key,  # 'key' is the name of the agent
        llm_config=gpt_config,
        system_message=system_message_1
    )
    agent_2 = autogen.AssistantAgent(
    name=key,  # 'key' is the name of the agent
    llm_config=gpt_config,
    system_message=system_message_1
)
    agent_3 = autogen.AssistantAgent(
    name=key,  # 'key' is the name of the agent
    llm_config=gpt_config,
    system_message=system_message_1
)
    Agents_1.append(agent_1)
    Agents_2.append(agent_2)
    Agents_3.append(agent_3)


user_proxy = autogen.UserProxyAgent(
   name="Government",
   system_message="A human admin. Interact with all other agents to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)

Agents_1.append(user_proxy)
Agents_2.append(user_proxy)
Agents_3.append(user_proxy)
print("Added user proxy")

# Create a group chat with all agents
groupchat_1 = autogen.GroupChat(
    agents = Agents_1,
    messages=[],
    max_round=100
)
groupchat_2 = autogen.GroupChat(
    agents = Agents_2,
    messages=[],
    max_round=100
)
groupchat_3 = autogen.GroupChat(
    agents = Agents_3,
    messages=[],
    max_round=100
)

manager_1 = autogen.GroupChatManager(groupchat=groupchat_1, llm_config=gpt_config)
manager_2 = autogen.GroupChatManager(groupchat=groupchat_2, llm_config=gpt_config)
manager_3 = autogen.GroupChatManager(groupchat=groupchat_3, llm_config=gpt_config)


Added user proxy


## Start Chat

In [13]:
# Round_1
user_proxy.initiate_chat(
    manager_1,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing community.
Urban Revitalization: Replacing an underutilized government building with residential housing can contribute to urban revitalization, potentially increasing the vibrancy and liveliness of the area.
Reduced Commute: For low-income individuals working in or near Kendall Square, living closer to work reduces commute time and transportation costs.
[Cons]
Safety concerns: low-income housing contributes to an increase in crime rates. This perception could affect the community's sense of security and could have social and economic repercussions. It may increase the risk of crime, violence, and drug, It will also lead to more homeless accumulate in the neighborhood.
Financial Implications: The cost of such a project and its long-term financial sustainability could be a concern. The project relies heavily on government subsidies by the government tax money for the low-income group
Community Displacement: The construction and subsequent changes in the neighborhood dynamic could lead to the displacement of current residents or local businesses.The introduction of a new demographic may alter the cultural and social fabric of the area, which could be viewed negatively by existing residents.
Land Value Fluctuations: The development may lead to a decrease in land values in the vicinity, particularly if there is public concern about potential negative impacts on the neighborhood. This could affect homeowners' investments and the local tax base. It will be harmful to the Kendall square's high-end branding
Schools and Public Services: An influx of families might place additional burdens on local schools and public services, which, if not properly managed, could lead to overcrowding and a decrease in the quality of these services. Hygenic and trash may also be an issue because we need more service budget to keep the neighborhood clean.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing com

In [ ]:
# Round_2
user_proxy.initiate_chat(
    manager_2,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing community.
Urban Revitalization: Replacing an underutilized government building with residential housing can contribute to urban revitalization, potentially increasing the vibrancy and liveliness of the area.
Reduced Commute: For low-income individuals working in or near Kendall Square, living closer to work reduces commute time and transportation costs.
[Cons]
Safety concerns: low-income housing contributes to an increase in crime rates. This perception could affect the community's sense of security and could have social and economic repercussions. It may increase the risk of crime, violence, and drug, It will also lead to more homeless accumulate in the neighborhood.
Financial Implications: The cost of such a project and its long-term financial sustainability could be a concern. The project relies heavily on government subsidies by the government tax money for the low-income group
Community Displacement: The construction and subsequent changes in the neighborhood dynamic could lead to the displacement of current residents or local businesses.The introduction of a new demographic may alter the cultural and social fabric of the area, which could be viewed negatively by existing residents.
Land Value Fluctuations: The development may lead to a decrease in land values in the vicinity, particularly if there is public concern about potential negative impacts on the neighborhood. This could affect homeowners' investments and the local tax base. It will be harmful to the Kendall square's high-end branding
Schools and Public Services: An influx of families might place additional burdens on local schools and public services, which, if not properly managed, could lead to overcrowding and a decrease in the quality of these services. Hygenic and trash may also be an issue because we need more service budget to keep the neighborhood clean.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing com

In [ ]:
# Round_3
user_proxy.initiate_chat(
    manager_3,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing community.
Urban Revitalization: Replacing an underutilized government building with residential housing can contribute to urban revitalization, potentially increasing the vibrancy and liveliness of the area.
Reduced Commute: For low-income individuals working in or near Kendall Square, living closer to work reduces commute time and transportation costs.
[Cons]
Safety concerns: low-income housing contributes to an increase in crime rates. This perception could affect the community's sense of security and could have social and economic repercussions. It may increase the risk of crime, violence, and drug, It will also lead to more homeless accumulate in the neighborhood.
Financial Implications: The cost of such a project and its long-term financial sustainability could be a concern. The project relies heavily on government subsidies by the government tax money for the low-income group
Community Displacement: The construction and subsequent changes in the neighborhood dynamic could lead to the displacement of current residents or local businesses.The introduction of a new demographic may alter the cultural and social fabric of the area, which could be viewed negatively by existing residents.
Land Value Fluctuations: The development may lead to a decrease in land values in the vicinity, particularly if there is public concern about potential negative impacts on the neighborhood. This could affect homeowners' investments and the local tax base. It will be harmful to the Kendall square's high-end branding
Schools and Public Services: An influx of families might place additional burdens on local schools and public services, which, if not properly managed, could lead to overcrowding and a decrease in the quality of these services. Hygenic and trash may also be an issue because we need more service budget to keep the neighborhood clean.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Pros]
Affordable Housing: The primary benefit is providing affordable housing options in an area that may otherwise be expensive. This can help diversify the community and provide living opportunities for lower-income families.
Economic Diversity: Introducing a mix of income levels can enrich the community's economic diversity, potentially leading to a more balanced local economy.
Community Services: Such developments often include additional community services and amenities, which can benefit both the new residents and the existing com

KeyboardInterrupt: 